In [8]:
import requests
import pandas as pd
import urllib.parse
import time
import os
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from dotenv import load_dotenv

# ✅ 환경변수 로딩
load_dotenv()
client_id = os.getenv("NAVER_CLIENT_ID")
client_secret = os.getenv("NAVER_CLIENT_SECRET")

# ✅ 경로
BASE_DIR = os.getcwd()
CHROME_DRIVER_PATH = os.path.join(BASE_DIR, "..", "driver", "chromedriver.exe")
COOKIE_PATH = os.path.join(BASE_DIR, "..", "driver", "kbland_cookies.json")

# ✅ 키워드 목록
keywords = [
    # ✅ 재건축·재개발·정비
    "재건축", "재개발", "리모델링", "정비사업", "정비구역", "재개발 추진", "조합 설립", "조합원 모집", "사업시행인가",
    "관리처분계획", "관리처분인가", "이주비", "이주계획", "현금청산", "도시정비사업", "공공재개발", "공공정비", "소규모재건축", "소규모정비사업",

    # ✅ 주택/아파트/시세
    "아파트 시세", "신축 아파트", "노후 아파트", "실거래가", "전세가율", "매매가", "입주 예정 아파트", "전세가 상승", "전세가 하락", "집값 전망",
    "주택 가격", "주택 시장", "미분양", "공급과잉", "수요부족", "초양극화 부동산", "로또 아파트", "똘똘한 한 채", "서울 입지 분석", "지역별 아파트 시세",

    # ✅ 청약/분양
    "청약", "아파트 청약", "분양", "분양권", "입주권", "청약 일정", "청약 경쟁률", "공공분양", "공공주택", "무순위 청약", "미계약분 청약", "사전청약", "생애최초 특별공급",

    # ✅ 정책·규제·세금
    "부동산 규제", "부동산 정책", "분양가 상한제", "전매제한", "대출 규제", "금리 인상", "금리 인하", "주택담보대출", "무주택자 혜택", "종부세",
    "양도세", "취득세", "보유세", "부동산 세금 회피", "LTV", "DTI", "DSR", "대출 한도", "실거주 요건", "전입신고 의무",

    # ✅ 사기/리스크
    "전세 사기", "깡통전세", "역전세", "전세난", "빌라왕", "부동산 사기", "불법중개", "허위매물", "시세 조작", "분양사기", "지연 입주", "계약 파기", "부동산 PF 리스크",

    # ✅ 지역별 시장
    "서울 집값", "강남 재건축", "강북 뉴타운", "경기도 아파트", "성남 재개발", "수도권 부동산", "인천 부동산", "대전 재개발", "부산 집값",
    "세종 부동산", "광주 부동산", "대구 아파트", "지방 부동산", "지방 미분양", "지방 재건축", "지방 재개발", "충청 부동산", "전라도 집값", "경상도 부동산",

    # ✅ 투자/분석/수요
    "부동산 투자", "실수요자", "갭투자", "N분의1 투자", "법인 매입", "부동산 컨설팅", "프롭테크", "AI 부동산 분석", "부동산 데이터", "부동산 빅데이터",
    "하우스 푸어", "영끌 투자", "패닉바잉", "투기과열지구", "투기지역", "조정대상지역", "주택 수요예측", "거래량 감소", "매물 잠김", "단기 급등 지역",

    # ✅ 고급 부동산·트렌드
    "하이엔드 아파트", "럭셔리 주택", "펜트하우스", "타워팰리스", "한남더힐", "힐스테이트", "자이", "푸르지오", "e편한세상", "래미안",
    "상업용 부동산", "지식산업센터", "도시재생", "스마트시티", "역세권 개발", "GTX 역세권", "3기 신도시", "복합개발", "역세권 청년주택", "브랜드 아파트"
]

all_news = []

# ✅ NAVER 뉴스 수집
def fetch_news(query, display=10, max_start=30):
    for start in range(1, max_start + 1, display):
        enc_query = urllib.parse.quote(query)
        url = f"https://openapi.naver.com/v1/search/news.json?query={enc_query}&display={display}&start={start}&sort=date"

        headers = {
            "X-Naver-Client-Id": client_id,
            "X-Naver-Client-Secret": client_secret
        }

        res = requests.get(url, headers=headers)
        if res.status_code != 200:
            print(f"❌ 오류 발생: {res.status_code} | {res.text}")
            break

        items = res.json().get("items", [])
        if not items:
            break

        for item in items:
            all_news.append({
                "title": item["title"].replace("<b>", "").replace("</b>", ""),
                "link": item["link"],
                "keyword": query,
                "source": "naver"
            })
        time.sleep(0.5)

# ✅ 호갱노노 뉴스
def crawl_hogangnono_news():
    print("📌 호갱노노 뉴스 수집 중...")

    options = Options()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(service=Service(CHROME_DRIVER_PATH), options=options)
    driver.get("https://hogangnono.com/news")
    time.sleep(3)

    print("✅ 페이지 타이틀:", driver.title)

    for _ in range(3):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    articles = soup.select("article.css-32e73x div.css-1k17v98")

    print("✅ 추출된 기사 수:", len(articles))

    for art in articles:
        title = art.get_text(strip=True)
        all_news.append({
            "title": title,
            "link": "https://hogangnono.com/news",
            "keyword": "호갱노노",
            "source": "hogangnono"
        })

    driver.quit()

# ✅ KB 커뮤니티
def crawl_kb_community():
    print("📌 KB부동산 커뮤니티 수집 중...")

    options = Options()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(service=Service(CHROME_DRIVER_PATH), options=options)
    
    # ✅ 쿠키 삽입 위해 도메인 먼저 열기
    driver.get("https://kbland.kr/ctm")
    time.sleep(2)

    # ✅ 쿠키 삽입
    if os.path.exists(COOKIE_PATH):
        with open(COOKIE_PATH, "r", encoding="utf-8") as f:
            cookies = json.load(f)
        for cookie in cookies:
            try:
                driver.add_cookie(cookie)
            except Exception as e:
                print("❌ 쿠키 삽입 실패:", cookie["name"], e)

    # ✅ 쿠키 적용 후 새로고침
    driver.refresh()
    time.sleep(3)

    print("✅ 커뮤니티 로그인 후 타이틀:", driver.title)

    for _ in range(3):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    cards = soup.select("div.card-wrap-commu")

    print("✅ 커뮤니티 글 수:", len(cards))

    for card in cards:
        try:
            title = card.get_text(strip=True)
            all_news.append({
                "title": title,
                "link": "https://kbland.kr/ctm",
                "keyword": "KB커뮤니티",
                "source": "kb"
            })
        except Exception as e:
            print("❌ 게시글 파싱 실패:", e)

    driver.quit()


# ✅ 실행
crawl_hogangnono_news()
crawl_kb_community()

for kw in keywords:
    print(f"🔍 '{kw}' 수집 중...")
    fetch_news(kw, display=10, max_start=100)

# ✅ 저장
df = pd.DataFrame(all_news)
output_path = os.path.join(BASE_DIR, "..", "data", "naver_realestate_news.csv")
df.to_csv(output_path, index=False, encoding="utf-8-sig")
print("✅ 저장 완료:", len(df), "건")


📌 호갱노노 뉴스 수집 중...
✅ 페이지 타이틀: 부동산 뉴스 | 아파트는 호갱노노
✅ 추출된 기사 수: 17
📌 KB부동산 커뮤니티 수집 중...
✅ 커뮤니티 로그인 후 타이틀: KB부동산 - 아파트 시세, 실거래가, 분양, 빌라시세, 예측시세
✅ 커뮤니티 글 수: 0
🔍 '재건축' 수집 중...
🔍 '재개발' 수집 중...
🔍 '리모델링' 수집 중...
🔍 '정비사업' 수집 중...
🔍 '정비구역' 수집 중...
🔍 '재개발 추진' 수집 중...
🔍 '조합 설립' 수집 중...
🔍 '조합원 모집' 수집 중...
🔍 '사업시행인가' 수집 중...
🔍 '관리처분계획' 수집 중...
🔍 '관리처분인가' 수집 중...
🔍 '이주비' 수집 중...
🔍 '이주계획' 수집 중...
🔍 '현금청산' 수집 중...
🔍 '도시정비사업' 수집 중...
🔍 '공공재개발' 수집 중...
🔍 '공공정비' 수집 중...
🔍 '소규모재건축' 수집 중...
🔍 '소규모정비사업' 수집 중...
🔍 '아파트 시세' 수집 중...
🔍 '신축 아파트' 수집 중...
🔍 '노후 아파트' 수집 중...
🔍 '실거래가' 수집 중...
🔍 '전세가율' 수집 중...
🔍 '매매가' 수집 중...
🔍 '입주 예정 아파트' 수집 중...
🔍 '전세가 상승' 수집 중...
🔍 '전세가 하락' 수집 중...
🔍 '집값 전망' 수집 중...
🔍 '주택 가격' 수집 중...
🔍 '주택 시장' 수집 중...
🔍 '미분양' 수집 중...
🔍 '공급과잉' 수집 중...
🔍 '수요부족' 수집 중...
🔍 '초양극화 부동산' 수집 중...
🔍 '로또 아파트' 수집 중...
🔍 '똘똘한 한 채' 수집 중...
🔍 '서울 입지 분석' 수집 중...
🔍 '지역별 아파트 시세' 수집 중...
🔍 '청약' 수집 중...
🔍 '아파트 청약' 수집 중...
🔍 '분양' 수집 중...
🔍 '분양권' 수집 중...
🔍 '입주권' 수집 중...
🔍 '청약 일정' 수집 중...
🔍 '청약 경쟁률' 수집 중...
🔍 '공공분양' 수집 중..